In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Import and Format Data

In [147]:
#Import datasets on the physical properties and make up of the semiconductors
sc_nums = pd.read_csv("superconductivty+data/train.csv")
sc_comp = pd.read_csv("superconductivty+data/unique_m.csv")

In [148]:
#Drop the columns that'll not be used for testing
sc_comp = sc_comp.drop(columns='material')
sc_comp = sc_comp.drop(columns='critical_temp')

In [149]:
#Combine data sets
sc = pd.concat([sc_nums, sc_comp], axis=1)
sc = sc.sample(frac=1).reset_index(drop=True)

In [150]:
#Split into train and test datasets
sc_X = sc.drop(columns='critical_temp')
sc_Y = sc['critical_temp']

sc_X = (sc_X - sc_X.mean()) / sc_X.std()
sc_Y = (sc_Y - sc_Y.mean()) / sc_Y.std()

sc_X_train = sc_X.loc[:14175]
sc_Y_train = sc_Y.loc[:14175]

sc_X_test = sc_X.loc[14176:21263]
sc_Y_test = sc_Y.loc[14176:21263]

In [151]:
#Function to find columns that are highly correlated to each other
def correlation(df, threshold):
    col_curr = set()
    correlation_matrix = df.corr()
    for i in range(len(df.columns)):
        for j in range(i):
            if (correlation_matrix.iloc[i,j]) > threshold:
                col_name = correlation_matrix.columns[i]
                col_curr.add(col_name)
    return col_curr      

In [152]:
#Drop columns that are highly correlated to get rid of redundant columns
corr_feature = correlation(sc_X_train, .90)
sc_X_train = sc_X_train.drop(columns=corr_feature, axis=1)
sc_X_test = sc_X_test.drop(columns=corr_feature, axis=1)

In [153]:
#Remove NaN values
def remove_na(df):
    for col in df.columns:
        if df[col].isna().any():
            df = df.drop(columns=col, axis=1)
    return df
            
sc_X_train = remove_na(sc_X_train)
sc_X_test = remove_na(sc_X_test)

## Feature Selection 

In [154]:
#Use sklearns mutual info regression to find which columns are most correlated with critical temperature
from sklearn.feature_selection import mutual_info_regression
mutual_info  = mutual_info_regression(sc_X_train, sc_Y_train)

In [156]:
#attatch mutual info regression scores to the column names
mutual_info = pd.Series(mutual_info)
mutual_info.index = sc_X_train.columns

In [92]:
len(sc_X_train.columns)

120

In [93]:
#See top 20 most correlated columns
mutual_info.sort_values(ascending=False).head(20).index.tolist()

['gmean_Density',
 'range_fie',
 'range_ElectronAffinity',
 'range_Density',
 'mean_ThermalConductivity',
 'gmean_ElectronAffinity',
 'range_atomic_mass',
 'gmean_ThermalConductivity',
 'mean_FusionHeat',
 'mean_ElectronAffinity',
 'mean_Density',
 'range_FusionHeat',
 'range_ThermalConductivity',
 'mean_atomic_mass',
 'wtd_std_ElectronAffinity',
 'mean_fie',
 'wtd_gmean_ThermalConductivity',
 'wtd_mean_fie',
 'wtd_range_FusionHeat',
 'entropy_ThermalConductivity']

In [18]:
#Create our training columns set
train_cols = mutual_info.sort_values(ascending=False).head().index.tolist()

## Linear Regression Model from Scratch

In [15]:
#Create dictionary to hold amount of columns and the corresponding R_sq score
col_num_score = {}

#Iterate through all amounts of columns, 1-120
for w in range(1, len(sc_X_train.columns)): 
    train_cols = mutual_info.sort_values(ascending=False).head(w).index.tolist()
    
    #Add a constant column to account for theta 0
    sc_X_train['constant'] = 1
    sc_X_test['constant'] = 1
    train_cols.append('constant')
    
    R_sq = 0

    alpha = 0.0001
    theta = np.random.randn(len(train_cols)) * 0.01

    sc_X_train_new = sc_X_train[train_cols]
    sc_X_test_new = sc_X_test[train_cols]

    y_train = sc_Y_train.values
    y_test = sc_Y_test.values
    
    max_r_sq = 0
    
    #Iterate through each set of columns 1000 times max
    for k in range(1000):
        
        #Shuffle datasets to account for ordering in the data set
        combined = pd.concat([sc_X_train_new, pd.Series(y_train, name='y')], axis=1)
        shuffled = combined.sample(frac=1).reset_index(drop=True)
        sc_X_train_shuffled = shuffled.drop(columns='y')
        y_train_shuffled = shuffled['y']

        prev_R_sq = R_sq
        
        #Schocastic Gradient Descent Algorithm
        for i in range(0, 300):
                h = np.dot(sc_X_train_shuffled.loc[i],theta)
                for j in range(0, len(train_cols)):
                    theta[j] = theta[j] - alpha*(h - y_train_shuffled[i])*sc_X_train_shuffled.loc[i][j]
        
        #Test the Mean absolute square, Mean square error, Root mean square error and R squared score
        h_test = np.dot(sc_X_test_new,theta)
        MAE = np.abs(h_test-y_test).sum()/len(y_test)
        MSE = pow(h_test-y_test,2).sum()/len(y_test)
        RMSE = np.sqrt(MSE)
        mean_y = np.mean(y_test)
        TSS = np.sum((y_test - mean_y) ** 2)
        R_sq = 1 - (np.sum((h_test - y_test) ** 2) / TSS)

        
        #Update the max score for the assocaited subset of columns
        if (R_sq > max_r_sq):
            max_r_sq = R_sq
            col_num_score[w] = max_r_sq
        
        #Stop model when the scores converge each iteration
        if (np.abs(R_sq - prev_R_sq) <= 0.000001):
            break
            
    #Print results for the subset of columns
    print ("Iteration: ", w)
    print("Test:")
    print ("MAE: ", MAE)
    print ("MSE: ", MSE)
    print ("RMSE: ", RMSE)
    print ("R Squared: ", max_r_sq)
    print ("")
    print('==================================================================================') 
        

Iteration:  1
Test:
MAE:  0.6962239355034276
MSE:  0.7167013582095683
RMSE:  0.8465821627045825
R Squared:  0.2938496448842316

Iteration:  2
Test:
MAE:  0.6421191968809137
MSE:  0.6473982758820573
RMSE:  0.8046106361974451
R Squared:  0.36192623709336447

Iteration:  3
Test:
MAE:  0.6250475674662753
MSE:  0.6255958994264341
RMSE:  0.7909462051406746
R Squared:  0.38346489927963545

Iteration:  4
Test:
MAE:  0.6258339655696827
MSE:  0.6198658448831945
RMSE:  0.7873155942080625
R Squared:  0.389146639395644

Iteration:  5
Test:
MAE:  0.6062425768182912
MSE:  0.5558929936803472
RMSE:  0.7455823185137556
R Squared:  0.4523461715557191

Iteration:  6
Test:
MAE:  0.5611892286182013
MSE:  0.5099593075212067
RMSE:  0.7141143518521432
R Squared:  0.4975605235645557

Iteration:  7
Test:
MAE:  0.5577461020350343
MSE:  0.48295490444806516
RMSE:  0.6949495697157205
R Squared:  0.5240016276297104

Iteration:  8
Test:
MAE:  0.5420381583826513
MSE:  0.45541799348292517
RMSE:  0.6748466444185117
R Squ

Iteration:  40
Test:
MAE:  0.4424522576125355
MSE:  0.33295940198926655
RMSE:  0.5770263442766427
R Squared:  0.6718428293727066

Iteration:  41
Test:
MAE:  0.4371739676725926
MSE:  0.32446310054805527
RMSE:  0.5696166259406894
R Squared:  0.6804468709476545

Iteration:  42
Test:
MAE:  0.43527082356034935
MSE:  0.31925315243652835
RMSE:  0.5650249131113851
R Squared:  0.6853934737526723

Iteration:  43
Test:
MAE:  0.4406831584421803
MSE:  0.3292800732082156
RMSE:  0.5738293066829331
R Squared:  0.675489009801484

Iteration:  44
Test:
MAE:  0.41759243614435515
MSE:  0.3034576420192041
RMSE:  0.5508698957278425
R Squared:  0.7009130750894317

Iteration:  45
Test:
MAE:  0.416875419894393
MSE:  0.30853534148176864
RMSE:  0.5554595768206437
R Squared:  0.6959828051279171

Iteration:  46
Test:
MAE:  0.414549795945653
MSE:  0.31179800788523593
RMSE:  0.5583887605291101
R Squared:  0.6926935870601231

Iteration:  47
Test:
MAE:  0.4422610180644129
MSE:  0.33735671043022625
RMSE:  0.580824164812

Iteration:  79
Test:
MAE:  0.3985887580161384
MSE:  0.290921836767276
RMSE:  0.5393717055679469
R Squared:  0.7132924421308053

Iteration:  80
Test:
MAE:  0.41746853385360705
MSE:  0.302574377445268
RMSE:  0.5500676117035687
R Squared:  0.7017842230884244

Iteration:  81
Test:
MAE:  0.3926077672133419
MSE:  0.319608902470034
RMSE:  0.565339634618018
R Squared:  0.7186699956678677

Iteration:  82
Test:
MAE:  0.39371198518963724
MSE:  0.28780001305726843
RMSE:  0.5364699554096841
R Squared:  0.717657671365961

Iteration:  83
Test:
MAE:  0.38981715334548184
MSE:  0.30669316665129015
RMSE:  0.5537988503520842
R Squared:  0.7210693655022917

Iteration:  84
Test:
MAE:  0.3905862134809421
MSE:  0.2871786949112464
RMSE:  0.5358905624390548
R Squared:  0.7195334417103133

Iteration:  85
Test:
MAE:  0.4192074513051307
MSE:  0.3110402313318013
RMSE:  0.5577098092483235
R Squared:  0.6937819604269553

Iteration:  86
Test:
MAE:  0.38967629787738156
MSE:  0.32642221989814824
RMSE:  0.571333720253013

Iteration:  118
Test:
MAE:  0.38724627673947515
MSE:  0.3170375836631183
RMSE:  0.5630609058202481
R Squared:  0.723824691019653

Iteration:  119
Test:
MAE:  0.3904734150089636
MSE:  0.28394396659767124
RMSE:  0.5328639287826408
R Squared:  0.7214626933951247



## Linear Regression Using Normal Equations

In [159]:
from sklearn.feature_selection import mutual_info_regression

def lin_reg_column_selector():
    mutual_info = mutual_info_regression(sc_X_train, sc_Y_train)
    mutual_info = pd.Series(mutual_info)
    mutual_info.index = sc_X_train.columns
    
    y_train = sc_Y_train.values
    y_test = sc_Y_test.values

    col_score = {}

    #Normal Equations
    for k in range (1, len(sc_X_train.columns)+1): 
        train_cols = mutual_info.sort_values(ascending=False).head(k).index.tolist()

        sc_X_train__ = sc_X_train[train_cols]
        sc_X_test__ = sc_X_test[train_cols]

            # Transpose the feature matrix
        sc_X_train_transpose = sc_X_train__.T

        # Calculate the dot product of the transposed matrix and the original matrix
        dot_product = np.dot(sc_X_train_transpose, sc_X_train__)

        # Compute the inverse of the dot product
        inverse_dot_product = np.linalg.inv(dot_product)

        # Calculate theta
        theta_new = np.dot(inverse_dot_product, np.dot(sc_X_train_transpose, sc_Y_train))
        
        h_test = np.dot(sc_X_test__,theta_new)
        mean_y = np.mean(y_test)
        TSS = np.sum((y_test - mean_y) ** 2)
        R_sq = 1 - (np.sum((h_test - y_test) ** 2) / TSS)

        col_score[k] = R_sq
    
    best_col_score = max(col_score, key=col_score.get)
    return (mutual_info.sort_values(ascending=False).head(best_col_score).index.tolist(), best_col_score, col_score[best_col_score])

In [160]:
#Picking mutual info regression run that has high accuracy at a low amount of features
col_scores = {}
smallest = 120
smallest_cols = []
for i in range(20):
    cols, num_cols, score = lin_reg_column_selector()
    col_scores[num_cols] = [cols, score]
    if num_cols <= smallest:
        smallest = num_cols
        smallest_cols = cols
    print(num_cols, " : ", score)

100  :  0.7413131359929842
109  :  0.742832075789313
92  :  0.7400971140015244
111  :  0.7434189677007532
92  :  0.7400590298730014
97  :  0.7403607401940642
103  :  0.7422707851054746
107  :  0.7428702920922803
118  :  0.7437384467125214
103  :  0.7419154602727747
112  :  0.7432840956144549
106  :  0.7421878416696884
105  :  0.7423181930649858
119  :  0.7437349142398881
97  :  0.7418545569812636
107  :  0.7429754520489329
97  :  0.74138745821764
107  :  0.7424606513364067
110  :  0.7432828499782866
96  :  0.7409111145914393


## SKLearn Linear Regression Model Comparison

In [161]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [167]:
train_cols = smallest_cols

sc_X_train['constant'] = 1
sc_X_test['constant'] = 1
train_cols.append('constant')

sc_X_train_new = sc_X_train[train_cols]
sc_X_test_new = sc_X_test[train_cols]

y_train = sc_Y_train.values
y_test = sc_Y_test.values
    

model = LinearRegression()
model.fit(sc_X_train_new, y_train)

y_pred = model.predict(sc_X_test_new)

In [168]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Mean Squared Error: 0.25659074842271123
R-squared: 0.7400279122613231


## Summary 

- 72% with 55 Features: Linear Regression Model from Scratch 
- 74% with 92 Features: Linear Regression Model using Normal Equations
- 74% with 92 Features:Linear Regression Model using SKLearn